# Project - Google Analytics Customer Revenue Preprocessing

## Presenting the initial data: 

<b>Data Fields: </b>

<b>fullVisitorIdv</b> - A unique identifier for each user of the Google Merchandise Store. <br>
<b>channelGrouping</b> - The channel via which the user came to the Store.<br>
<b>date</b> - The date on which the user visited the Store.<br>
<b>device </b>- The specifications for the device used to access the Store.<br>
<b>geoNetwork</b> - This section contains information about the geography of the user.<br>
<b>sessionId</b> - A unique identifier for this visit to the store.<br>
<b>socialEngagementType</b> - Engagement type, either "Socially Engaged" or "Not Socially Engaged".<br>
<b>totals</b> - This section contains aggregate values across the session.<br>
<b>trafficSource</b> - This section contains information about the Traffic Source from which the session originated.<br>
<b>visitId</b> - An identifier for this session. This is part of the value usually stored as the _utmb cookie. This is only unique to the user. For a completely unique ID, you should use a combination of fullVisitorId and visitId.<br>
<b>visitNumber</b> - The session number for this user. If this is the first session, then this is set to 1.<br>
<b>visitStartTime</b> - The timestamp (expressed as POSIX time).<br>

# Objectives: 

The main objectives of this project are :

* Load the data so everything is in tabular format (some columns contain JSON so it you will need to find ways to separate those into independent columns)
* Identify the variables that need special processing (removing or infering missing values, removing columns that don't contain useful information)
* Run visualizations to better understand the data

## Importing necessary libraries

In [1]:
!pip install s3fs

In [ ]:
#import libraries
import os
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from pandas.io.json import json_normalize
from datetime import datetime
from sklearn import preprocessing

plt.style.use('fivethirtyeight')

In [2]:
#process the columns in json formaat
columns = ['device', 'geoNetwork', 'totals', 'trafficSource']

def json_read(df):
    data_frame = df
    p = 0.07
    df = pd.read_csv(data_frame, 
                     converters={column: json.loads for column in columns},
                     dtype={'fullVisitorId': 'str'},
                     skiprows=lambda i: i>0 and random.random() > p)
    
    for column in columns: 
        column_as_df = json_normalize(df[column]) 
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns] 
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
           
    print(f"Loaded {os.path.basename(data_frame)}. Shape: {df.shape}")
    return df

In [ ]:
#import data
df = json_read("s3://full-stack-bigdata-datasets/Machine Learning Supervisé/projects/preprocessing_linear_models/Google_dataset.csv") 

In [ ]:
pd.set_option('display.max_columns', 500)

In [ ]:
df.head()

In [ ]:
#check for missing values
def missing_values(data):
    total = data.isnull().sum().sort_values(ascending = False) # getting the sum of null values and ordering
    percent = (data.isnull().sum() / data.isnull().count() * 100 ).sort_values(ascending = False) #getting the percent and order of null
    df = pd.concat([total, percent], axis=1, keys=['Total', 'Percent']) # Concatenating the total and percent
    print("Total columns at least one missing values: ")
    print (df[~(df['Total'] == 0)]) # Returning values of nulls different of 0
    
    return 

In [ ]:
missing_values(df) 

In [ ]:
#check datatypes
print(df.info())
print("\n \n The distribution of columns across types is the following : \n {}".format(df.dtypes.value_counts()))

In [ ]:
#deal with the date
from datetime import datetime

def date_process(df):
    df["date"] = pd.to_datetime(df["date"], format="%Y%m%d")
    df["_weekday"] = df['date'].dt.weekday
    df["_day"] = df['date'].dt.day
    df["_month"] = df['date'].dt.month
    df["_year"] = df['date'].dt.year
    df['_visitHour'] = (df['visitStartTime'].apply(lambda x: str(datetime.fromtimestamp(x).hour))).astype(int)
    
    return df

In [ ]:
df_train = date_process(df)
df_train.head(n=2)

In [ ]:
#check for columns of constants
discovering_consts = [col for col in df_train.columns if df_train[col].nunique() == 1]
df_train = df_train.drop(discovering_consts, axis = 1)

print("Columns with just one value: ", len(discovering_consts), "columns \n")
print("Name of constant columns: \n")
for col in discovering_consts :
    print(col)

In [ ]:
#check number of unique values per column
def knowingData(df, limit=3):
    for column in df.columns:
        print("_________________________________")
        print("Name of column ", column, ': \n', "Uniques: ", df[column].unique()[:limit], "\n",
              " | ## Total nulls: ", (round(df[column].isnull().sum() / len(df[column]) * 100,2)),
              " | ## Total unique values: ", df_train.nunique()[column])
        
    )
        print("_________________________________")

In [ ]:
knowingData(df_train)

In [ ]:
#drop unnecessary columns
to_drop = ['trafficSource.adwordsClickInfo.gclId', 'trafficSource.campaign',
           'trafficSource.adwordsClickInfo.page', 'trafficSource.referralPath', 'trafficSource.adwordsClickInfo.slot',
           'trafficSource.adContent', 'trafficSource.keyword']

In [ ]:
df_train.drop(to_drop, axis=1, inplace=True)

In [ ]:
print("Total features dropped: ", len(to_drop))
print("Shape after dropping: ", df_train.shape)

Total features dropped:  7
Shape after dropping:  (63583, 28)


In [ ]:
#handle missing values
def NumericalColumns(df):
    df['totals.pageviews'].fillna(1, inplace=True)
    df["totals.transactionRevenue"] = df["totals.transactionRevenue"].fillna(0.0).astype(float)
    df['totals.pageviews'] = df['totals.pageviews'].astype(int)
    df["totals.hits"] = df["totals.hits"].astype(float)

    return df

In [ ]:
#normalize data
def Normalizing(df):
    # Use MinMaxScaler to normalize the column
    df["totals.hits"] =  (df['totals.hits'] - min(df['totals.hits'])) / (max(df['totals.hits'])  - min(df['totals.hits']))
    # normalizing the transaction Revenue
    df['totals.transactionRevenue'] = df_train['totals.transactionRevenue'].apply(lambda x: np.log10(x+1))
    # return the modified df
    return df 

In [ ]:
df_train = NumericalColumns(df_train)
df_train = Normalizing(df_train)

In [ ]:
dummy_feaures =['channelGrouping', 'device.browser', 'device.deviceCategory', 'geoNetwork.city', 'device.operatingSystem', 
                'trafficSource.medium', 'trafficSource.source',
                'geoNetwork.continent', 'geoNetwork.country', 'geoNetwork.metro', 'geoNetwork.networkDomain', 'geoNetwork.region', 
                'geoNetwork.subContinent']


numericals = ['totals.visits', '_visitHour', '_day', '_month', '_weekday']

In [ ]:
#check stats and visualize revenue
print("Transaction Revenue Min Value: ", 
      df_train[df_train['totals.transactionRevenue'] > 0]["totals.transactionRevenue"].min())
print("Transaction Revenue Mean Value: ", 
      df_train[df_train['totals.transactionRevenue'] > 0]["totals.transactionRevenue"].mean())
print("Transaction Revenue Median Value: ", 
      df_train[df_train['totals.transactionRevenue'] > 0]["totals.transactionRevenue"].median())
print("Transaction Revenue Max Value: ", 
      df_train[df_train['totals.transactionRevenue'] > 0]["totals.transactionRevenue"].max())

plt.figure(figsize=(14,5))

plt.subplot(1,1,1)

ax = sns.distplot(np.log(df_train[df_train['totals.transactionRevenue'] > 0]["totals.transactionRevenue"]), bins=40, kde=True)
ax.set_xlabel('Transaction RevenueLog', fontsize=15)
ax.set_ylabel('Distribution', fontsize=15)
ax.set_title("Distribution of Revenue Log", fontsize=20)

plt.show()

In [ ]:
print("Browser usage: ")
print(df_train['device.browser'].value_counts()[:7] )

fig, ax = plt.subplots(1,1,figsize=(14,6))

sns.countplot(df_train[df_train['device.browser']\
                       .isin(df_train['device.browser']\
                             .value_counts()[:10].index.values)]['device.browser'], palette="hls")
total = len(df_train['device.browser'])
for p in ax.patches:
        percentage = '{:.1f}%'.format(100 * p.get_height()/total)
        x = p.get_x() + 0.1
        y = p.get_y() + p.get_height() + 0.5
        ax.annotate(percentage, (x, y))
plt.title("TOP 10 Most Frequent Browsers", fontsize=20)
plt.xlabel("Browser Names", fontsize=16)
plt.ylabel("Count", fontsize=16)
plt.xticks(rotation=45)

plt.show()

In [ ]:
#plot revenue against browser
plt.figure(figsize=(13,6))

g1 = sns.boxenplot(x='device.browser', y='totals.transactionRevenue', 
                   data=df_train[(df_train['device.browser'].isin((df_train['device.browser'].value_counts()[:10].index.values))) &
                                  df_train['totals.transactionRevenue'] > 0])
g1.set_title('Browsers Name by Transactions Revenue', fontsize=20)
g1.set_xticklabels(g1.get_xticklabels(),rotation=45)
g1.set_xlabel('Device Names', fontsize=18)
g1.set_ylabel('Trans Revenue(log) Dist', fontsize=18)

plt.show()

In [ ]:
#visualize acquisition channel
print("Percentage of Channel Grouping used: ")
print((df_train['channelGrouping'].value_counts()[:5]))

fig, ax = plt.subplots(1,1,figsize=(14,7))

sns.countplot(df_train["channelGrouping"], palette="hls")

total = len(df_train['channelGrouping'])
for p in ax.patches:
        percentage = '{:.1f}%'.format(100 * p.get_height()/total)
        x = p.get_x() + 0.1
        y = p.get_y() + p.get_height() + 0.5
        ax.annotate(percentage, (x, y))

plt.title("Channel Grouping Count", fontsize=20)
plt.xlabel("Channel Grouping Name", fontsize=18)
plt.ylabel("Count", fontsize=18)

plt.show()

In [ ]:
#channel grouping by browsers
crosstab_eda = pd.crosstab(index=df_train['channelGrouping'], normalize=True,
                           columns=df_train[df_train['device.browser'].isin(df_train['device.browser']\
                                                                            .value_counts()[:5].index.values)]['device.browser'])
crosstab_eda.plot(kind="bar",
                 figsize=(14,7),
                 stacked=True)
plt.title("Channel Grouping % for which Browser", fontsize=20)
plt.xlabel("The Channel Grouping Name", fontsize=18)
plt.ylabel("Count", fontsize=18)
plt.xticks(rotation=0)
plt.show()

In [ ]:
#operating system
print("Percentage of Operational Systems: ")
print(df_train['device.operatingSystem'].value_counts()[:5])

fig, ax = plt.subplots(figsize=(14,14))

sns.countplot(df_train["device.operatingSystem"], palette="hls")
total = len(df_train['device.operatingSystem'])
for p in ax.patches:
        percentage = '{:.1f}%'.format(100 * p.get_height()/total)
        x = p.get_x() + 0.1
        y = p.get_y() + p.get_height() + 0.5
        ax.annotate(percentage, (x, y))
plt.title("Operational System used Count", fontsize=20)
plt.xlabel("Operational System Name", fontsize=16)
plt.ylabel("OS Count", fontsize=16)
plt.xticks(rotation=45)

plt.show()

In [ ]:
#browser by operating system
crosstab_eda = pd.crosstab(index=df_train[df_train['device.operatingSystem']\
                                          .isin(df_train['device.operatingSystem']\
                                                .value_counts()[:6].index.values)]['device.operatingSystem'], 
                    
                           columns=df_train[df_train['device.browser'].isin(df_train['device.browser']\
                                                                            .value_counts()[:5].index.values)]['device.browser'])
crosstab_eda.plot(kind="bar",
                 figsize=(14,7),
                 stacked=True)
plt.title("Most frequent OS's by Browsers of users", fontsize=22)
plt.xlabel("Operational System Name", fontsize=19)
plt.ylabel("Count OS", fontsize=19)
plt.xticks(rotation=0)

plt.show()

In [ ]:
#transaction revenue by OS
(sns.FacetGrid(df_train[(df_train['device.operatingSystem']\
                        .isin(df_train['device.operatingSystem']\
                              .value_counts()[:6].index.values)) & df_train['totals.transactionRevenue'] > 0],
               hue='device.operatingSystem', height=5, aspect=2)
  .map(sns.kdeplot, 'totals.transactionRevenue', shade=True)
 .add_legend()
)
plt.show()

In [ ]:
#device category
print("Percentage of Devices: ")
print(round(df_train['device.deviceCategory'].value_counts() / len(df_train['device.deviceCategory']) * 100, 2))

fig , ax = plt.subplots(1,2,figsize=(14,5))

sns.countplot(df_train["device.deviceCategory"], palette="hls", ax = ax[0])
total = len(df_train['device.deviceCategory'])
for p in ax[0].patches:
        percentage = '{:.1f}%'.format(100 * p.get_height()/total)
        x = p.get_x() + 0.1
        y = p.get_y() + p.get_height() + 0.5
        ax[0].annotate(percentage, (x, y))
plt.title("Device Category Count", fontsize=20)
plt.xlabel("Device Category", fontsize=18)
plt.ylabel("Count", fontsize=16)
plt.xticks(fontsize=18)

sns.boxenplot(x="device.deviceCategory", y = 'totals.transactionRevenue', 
              data=df_train[df_train['totals.transactionRevenue'] > 0], palette="hls", ax=ax[1])
plt.title("Device Category Revenue Distribuition", fontsize=20)
plt.xlabel("Device Category", fontsize=18)
plt.ylabel("Revenue(Log)", fontsize=16)
plt.xticks(fontsize=18)

plt.subplots_adjust(hspace = 0.9, wspace = 0.5)

plt.show()

In [ ]:
#device comparison
(sns.FacetGrid(df_train[df_train['totals.transactionRevenue'] > 0],
               hue='device.deviceCategory', height=5, aspect=2)
  .map(sns.kdeplot, 'totals.transactionRevenue', shade=True)
 .add_legend()
)
plt.show()

In [ ]:
# device and browsers
crosstab_eda = pd.crosstab(index=df_train['device.deviceCategory'],
                           columns=df_train[df_train['device.operatingSystem']\
                                            .isin(df_train['device.operatingSystem']\
                                                  .value_counts()[:6].index.values)]['device.operatingSystem'])

crosstab_eda.plot(kind="bar",
                 figsize=(14,7),
                 stacked=True)
plt.title("Most frequent OS's by Device Categorys of users", fontsize=22) 
plt.xlabel("Device Name", fontsize=19)      
plt.ylabel("Count Device x OS", fontsize=19)                
plt.xticks(rotation=0)                  


plt.show()

In [ ]:
#by subcontinent
print("Description of SubContinent count: ")
print(df_train['geoNetwork.subContinent'].value_counts()[:8])

fig, ax = plt.subplots(figsize=(16,7))

sns.countplot(df_train[df_train['geoNetwork.subContinent']\
                       .isin(df_train['geoNetwork.subContinent']\
                             .value_counts()[:15].index.values)]['geoNetwork.subContinent'], palette="hls") 
total = len(df_train['geoNetwork.subContinent'])
for p in ax.patches:
        percentage = '{:.1f}%'.format(100 * p.get_height()/total)
        x = p.get_x() + 0.1
        y = p.get_y() + p.get_height() + 0.5
        ax.annotate(percentage, (x, y))
plt.title("TOP 15 most frequent SubContinents", fontsize=20)
plt.xlabel("subContinent Names", fontsize=18)
plt.ylabel("SubContinent Count", fontsize=18)
plt.xticks(rotation=45)

plt.show()

In [ ]:
#subcontinent by browser
crosstab_eda = pd.crosstab(index=df_train[df_train['geoNetwork.subContinent']\
                                          .isin(df_train['geoNetwork.subContinent']\
                                                .value_counts()[:10].index.values)]['geoNetwork.subContinent'], 
                           
                           columns=df_train[df_train['device.browser'].isin(df_train['device.browser']\
                                                                            .value_counts()[:5].index.values)]['device.browser'])

crosstab_eda.plot(kind="bar", 
                 figsize=(16,7), 
                 stacked=True) 
plt.title("TOP 10 Most frequent Subcontinents by Browsers used", fontsize=22) 
plt.xlabel("Subcontinent Name", fontsize=19) 
plt.ylabel("Count Subcontinent", fontsize=19)
plt.xticks(rotation=45)
plt.legend(loc=1, prop={'size': 12})

plt.show()